<a href="https://colab.research.google.com/github/gauthambageshapur/MLE-codes/blob/main/GenAI_NPPE_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()  # choose kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"iitm21f3002294","key":"0b6b53fae70f9666fd83d61310afe09b"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:
!kaggle competitions download -c sep-25-dl-gen-ai-nppe-1

 96% 501M/522M [00:01<00:00, 331MB/s]
100% 522M/522M [00:01<00:00, 428MB/s]


In [4]:
!unzip /content/sep-25-dl-gen-ai-nppe-1.zip -d /content/


Streaming output truncated to the last 5000 lines.
  inflating: /content/face_dataset/train/29708.jpg  
  inflating: /content/face_dataset/train/29709.jpg  
  inflating: /content/face_dataset/train/29710.jpg  
  inflating: /content/face_dataset/train/29711.jpg  
  inflating: /content/face_dataset/train/29712.jpg  
  inflating: /content/face_dataset/train/29713.jpg  
  inflating: /content/face_dataset/train/29714.jpg  
  inflating: /content/face_dataset/train/29715.jpg  
  inflating: /content/face_dataset/train/29716.jpg  
  inflating: /content/face_dataset/train/29717.jpg  
  inflating: /content/face_dataset/train/29718.jpg  
  inflating: /content/face_dataset/train/29719.jpg  
  inflating: /content/face_dataset/train/29720.jpg  
  inflating: /content/face_dataset/train/29721.jpg  
  inflating: /content/face_dataset/train/29722.jpg  
  inflating: /content/face_dataset/train/29723.jpg  
  inflating: /content/face_dataset/train/29724.jpg  
  inflating: /content/face_dataset/train/29725.j

In [7]:
!ls -lh /content/face_dataset/


total 2.4M
-rw-r--r-- 1 root root   84K Nov  6 14:42 sample_submission.csv
drwxr-xr-x 2 root root  260K Nov  9 06:31 test
-rw-r--r-- 1 root root  169K Nov  6 14:42 test.csv
drwxr-xr-x 2 root root 1020K Nov  9 06:31 train
-rw-r--r-- 1 root root  905K Nov  6 14:42 train.csv


In [6]:
import os
os.getcwd()

'/content'

In [9]:
import os, time, random, math
from typing import Tuple, Dict

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models

# -------------------------
# Config (EDIT THESE)
# -------------------------
CSV_PATH    = "/content/face_dataset/train.csv"      # <-- change
IMAGE_ROOT  = "/content/face_dataset/"    # <-- change (folder that contains 'train/' referenced in CSV)
IMG_SIZE    = 224
BATCH_SIZE  = 64
EPOCHS      = 12
LR          = 3e-4
WEIGHT_DECAY= 1e-4
LAMBDA_AGE  = 1.0        # weight on age MSE (tune 0.5~2.0)
VAL_RATIO   = 0.2
SEED        = 42
NUM_WORKERS = 2
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
# -------------------------
# Reproducibility
# -------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

In [11]:
# Dataset
# -------------------------
class FaceMultiTaskDataset(Dataset):
    """
    Expects CSV columns: id, full_path, gender, age
    Returns: image_tensor, gender_label (int64), age (float32)
    If age_mean/std given, .age_norm returns standardized age for training.
    """
    def __init__(self, csv_path, image_root=".", img_size=224, augment=False,
                 age_mean=None, age_std=None):
        self.df = pd.read_csv(csv_path)
        self.image_root = image_root
        self.age_mean = age_mean
        self.age_std  = age_std

        if augment:
            self.tfms = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(10),
                transforms.ColorJitter(0.1, 0.1, 0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485,0.456,0.406],
                                     std=[0.229,0.224,0.225]),
            ])
        else:
            self.tfms = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485,0.456,0.406],
                                     std=[0.229,0.224,0.225]),
            ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_root, row["full_path"])
        with Image.open(img_path) as im:
            img = im.convert("RGB")
        x = self.tfms(img)
        gender = int(row["gender"])
        age = float(row["age"])
        age_norm = age
        if self.age_mean is not None and self.age_std is not None and self.age_std > 0:
            age_norm = (age - self.age_mean) / self.age_std
        return x, torch.tensor(gender, dtype=torch.long), torch.tensor(age, dtype=torch.float32), torch.tensor(age_norm, dtype=torch.float32)


In [12]:
# -------------------------
# Stratified split by gender (no sklearn)
# -------------------------
def stratified_split_by_gender(csv_path, val_ratio=0.2, seed=42):
    df = pd.read_csv(csv_path)
    rng = np.random.default_rng(seed)
    train_idx, val_idx = [], []
    for g, gdf in df.groupby("gender"):
        idx = gdf.index.to_numpy()
        rng.shuffle(idx)
        n_val = int(len(idx) * val_ratio)
        val_idx.extend(idx[:n_val])
        train_idx.extend(idx[n_val:])
    return sorted(train_idx), sorted(val_idx), df

train_idx, val_idx, df_all = stratified_split_by_gender(CSV_PATH, VAL_RATIO, SEED)

# Class weights (gender imbalance)
gender_counts = df_all["gender"].value_counts().sort_index().to_numpy().astype(float)
class_weights = (gender_counts.mean() / gender_counts).astype("float32")

# Age normalization from TRAIN only
age_mean = df_all.iloc[train_idx]["age"].mean()
age_std  = df_all.iloc[train_idx]["age"].std()

print(f"Gender counts: {gender_counts} -> class weights: {class_weights}")
print(f"Age mean/std (train only): {age_mean:.3f}/{age_std:.3f}")


Gender counts: [ 8624. 26084.] -> class weights: [2.0122912  0.66531205]
Age mean/std (train only): 37.422/16.606


In [13]:
# -------------------------
# Datasets / Loaders
# -------------------------
full_train = FaceMultiTaskDataset(CSV_PATH, IMAGE_ROOT, IMG_SIZE, augment=True,
                                  age_mean=age_mean, age_std=age_std)
full_val   = FaceMultiTaskDataset(CSV_PATH, IMAGE_ROOT, IMG_SIZE, augment=False,
                                  age_mean=age_mean, age_std=age_std)

train_ds = Subset(full_train, train_idx)
val_ds   = Subset(full_val,   val_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)

In [14]:
# -------------------------
# Models
# -------------------------
class SimpleCNN(nn.Module):
    """ Small CNN from scratch with shared trunk + two heads. """
    def __init__(self, num_gender_classes=2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 112x112

            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 56x56

            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 28x28

            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 14x14

            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1,1))       # 256-d feature
        )
        self.dropout = nn.Dropout(0.3)
        self.gender_head = nn.Linear(256, num_gender_classes)
        self.age_head    = nn.Linear(256, 1)

    def forward(self, x):
        f = self.features(x)
        f = f.view(x.size(0), -1)
        f = self.dropout(f)
        gender_logits = self.gender_head(f)
        age_norm_pred = self.age_head(f).squeeze(1)  # normalized age
        return gender_logits, age_norm_pred

class ResNet18MultiTask(nn.Module):
    """ Pretrained ResNet18 backbone + two heads. """
    def __init__(self, num_gender_classes=2, pretrained=True, freeze_backbone=False):
        super().__init__()
        self.backbone = models.resnet18(pretrained=pretrained)
        in_feats = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        self.gender_head = nn.Linear(in_feats, num_gender_classes)
        self.age_head    = nn.Linear(in_feats, 1)

        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

    def forward(self, x):
        f = self.backbone(x)                 # [B, in_feats]
        gender_logits = self.gender_head(f)  # [B,2]
        age_norm_pred = self.age_head(f).squeeze(1)
        return gender_logits, age_norm_pred

In [15]:
# -------------------------
# Losses / Optim / Sched
# -------------------------
ce_weights = torch.tensor(class_weights, dtype=torch.float, device=DEVICE)
crit_gender = nn.CrossEntropyLoss(weight=ce_weights)
crit_age    = nn.MSELoss()

def multitask_loss(gender_logits, gender_targets, age_norm_pred, age_norm_targets):
    loss_g = crit_gender(gender_logits, gender_targets)
    loss_a = crit_age(age_norm_pred, age_norm_targets)
    return loss_g + LAMBDA_AGE * loss_a, loss_g.item(), loss_a.item()

In [16]:

# -------------------------
# Metrics
# -------------------------
from sklearn.metrics import f1_score, accuracy_score

@torch.no_grad()
def compute_metrics(gender_logits, gender_targets, age_norm_pred, age_norm_targets,
                    age_mean, age_std) -> Dict[str, float]:
    # gender
    preds = gender_logits.argmax(1).cpu().numpy()
    t     = gender_targets.cpu().numpy()
    acc = accuracy_score(t, preds)
    f1  = f1_score(t, preds, average="binary")

    # age back to original scale
    age_pred = (age_norm_pred.cpu().numpy() * age_std) + age_mean
    age_true = (age_norm_targets.cpu().numpy() * age_std) + age_mean
    mae = np.mean(np.abs(age_pred - age_true))
    rmse = np.sqrt(np.mean((age_pred - age_true)**2))
    return {"acc": acc, "f1": f1, "mae": mae, "rmse": rmse}


In [18]:
# -------------------------
# Train / Eval loops
# -------------------------
def run_epoch(model, loader, optimizer=None, device=DEVICE, desc="train"):
    is_train = optimizer is not None
    model.train(is_train)

    total = 0
    sum_loss = 0.0
    sum_loss_g = 0.0
    sum_loss_a = 0.0

    all_logits = []
    all_g = []
    all_age_norm_pred = []
    all_age_norm_true = []

    for x, g, age, age_norm in loader:
        x, g, age_norm = x.to(device), g.to(device), age_norm.to(device)

        gender_logits, age_norm_pred = model(x)
        loss, lg, la = multitask_loss(gender_logits, g, age_norm_pred, age_norm)

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        bs = x.size(0)
        total += bs
        sum_loss   += loss.item() * bs
        sum_loss_g += lg * bs
        sum_loss_a += la * bs

        all_logits.append(gender_logits.detach().cpu())
        all_g.append(g.detach().cpu())
        all_age_norm_pred.append(age_norm_pred.detach().cpu())
        all_age_norm_true.append(age_norm.detach().cpu())

    # concat and compute metrics
    logits = torch.cat(all_logits, 0)
    gtrue  = torch.cat(all_g, 0)
    apred  = torch.cat(all_age_norm_pred, 0)
    atrue  = torch.cat(all_age_norm_true, 0)

    metrics = compute_metrics(logits, gtrue, apred, atrue, age_mean, age_std)

    avg = {
        "loss": sum_loss/total,
        "loss_gender": sum_loss_g/total,
        "loss_age": sum_loss_a/total,
        **metrics
    }
    return avg

def train_and_eval(model, name="model"):
    model = model.to(DEVICE)
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                                  lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

    best_key = "mae"   # choose your selection metric (lower is better)
    best_val = float("inf")
    best_path = f"{name}_best.pt"

    for epoch in range(1, EPOCHS+1):
        t0 = time.time()
        tr = run_epoch(model, train_loader, optimizer, DEVICE, desc="train")
        va = run_epoch(model, val_loader,   optimizer=None, device=DEVICE, desc="val")
        scheduler.step()

        improved = va[best_key] < best_val
        if improved:
            best_val = va[best_key]
            torch.save(model.state_dict(), best_path)

        dt = time.time() - t0
        print(f"[{name}][{epoch:02d}/{EPOCHS}] {dt:.1f}s  "
              f"train: loss={tr['loss']:.4f} G={tr['loss_gender']:.4f} A={tr['loss_age']:.4f}  "
              f"acc={tr['acc']:.3f} f1={tr['f1']:.3f} mae={tr['mae']:.2f} rmse={tr['rmse']:.2f} | "
              f"val: loss={va['loss']:.4f} G={va['loss_gender']:.4f} A={va['loss_age']:.4f}  "
              f"acc={va['acc']:.3f} f1={va['f1']:.3f} mae={va['mae']:.2f} rmse={va['rmse']:.2f} "
              f"{'**BEST**' if improved else ''}")

    print(f"[{name}] best {best_key}={best_val:.3f} -> saved {best_path}")
    return model, best_path


In [19]:
# -------------------------
# 1) Train Simple CNN (from scratch)
# -------------------------
simple_model = SimpleCNN()
simple_model, simple_ckpt = train_and_eval(simple_model, name="simple_cnn")

[simple_cnn][01/12] 179.6s  train: loss=1.6242 G=0.6700 A=0.9542  acc=0.622 f1=0.724 mae=13.13 rmse=16.22 | val: loss=1.4887 G=0.6356 A=0.8532  acc=0.586 f1=0.664 mae=12.24 rmse=15.34 **BEST**
[simple_cnn][02/12] 174.8s  train: loss=1.4544 G=0.6113 A=0.8431  acc=0.708 f1=0.795 mae=12.11 rmse=15.25 | val: loss=1.3703 G=0.5749 A=0.7954  acc=0.780 f1=0.856 mae=11.87 rmse=14.81 **BEST**
[simple_cnn][03/12] 174.1s  train: loss=1.3360 G=0.5611 A=0.7749  acc=0.736 f1=0.814 mae=11.51 rmse=14.62 | val: loss=1.2361 G=0.5167 A=0.7195  acc=0.786 f1=0.852 mae=10.92 rmse=14.09 **BEST**
[simple_cnn][04/12] 174.2s  train: loss=1.2253 G=0.5187 A=0.7066  acc=0.761 f1=0.831 mae=10.92 rmse=13.96 | val: loss=1.2353 G=0.4879 A=0.7474  acc=0.809 f1=0.868 mae=10.90 rmse=14.36 **BEST**
[simple_cnn][05/12] 175.7s  train: loss=1.1383 G=0.4698 A=0.6685  acc=0.794 f1=0.856 mae=10.61 rmse=13.58 | val: loss=1.0538 G=0.4493 A=0.6045  acc=0.860 f1=0.908 mae=10.07 rmse=12.91 **BEST**
[simple_cnn][06/12] 173.5s  train: 

In [20]:
# -------------------------
# 2) Train Finetuned ResNet-18
#    (try freeze_backbone=True for linear-probe, then unfreeze to finetune)
# -------------------------
resnet_model = ResNet18MultiTask(pretrained=True, freeze_backbone=False)
resnet_model, resnet_ckpt = train_and_eval(resnet_model, name="resnet18_ft")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 199MB/s]


[resnet18_ft][01/12] 183.8s  train: loss=0.6678 G=0.2520 A=0.4157  acc=0.904 f1=0.934 mae=8.05 rmse=10.71 | val: loss=0.6482 G=0.1786 A=0.4696  acc=0.931 f1=0.953 mae=9.18 rmse=11.38 **BEST**
[resnet18_ft][02/12] 183.6s  train: loss=0.4755 G=0.1798 A=0.2958  acc=0.938 f1=0.958 mae=6.74 rmse=9.03 | val: loss=0.5111 G=0.1713 A=0.3398  acc=0.951 f1=0.968 mae=6.93 rmse=9.68 **BEST**
[resnet18_ft][03/12] 182.6s  train: loss=0.4127 G=0.1480 A=0.2646  acc=0.948 f1=0.965 mae=6.36 rmse=8.54 | val: loss=0.4536 G=0.1814 A=0.2722  acc=0.955 f1=0.970 mae=6.39 rmse=8.66 **BEST**
[resnet18_ft][04/12] 182.4s  train: loss=0.3603 G=0.1299 A=0.2305  acc=0.956 f1=0.970 mae=5.93 rmse=7.97 | val: loss=0.4475 G=0.1558 A=0.2917  acc=0.943 f1=0.961 mae=6.57 rmse=8.97 
[resnet18_ft][05/12] 183.3s  train: loss=0.3131 G=0.1102 A=0.2029  acc=0.961 f1=0.974 mae=5.58 rmse=7.48 | val: loss=0.4322 G=0.1674 A=0.2648  acc=0.945 f1=0.963 mae=6.33 rmse=8.55 **BEST**
[resnet18_ft][06/12] 182.0s  train: loss=0.2774 G=0.0929

In [ ]:
# -------------------------
# Inference helper (returns gender prob + age in original scale)
# -------------------------
@torch.no_grad()
def predict_image(model, image_path, device=DEVICE):
    model.eval()
    tfm = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225]),
    ])
    with Image.open(image_path) as im:
        img = im.convert("RGB")
    x = tfm(img).unsqueeze(0).to(device)
    glogits, age_norm = model(x)
    gprob = F.softmax(glogits, dim=1).squeeze(0).cpu().numpy()
    age_pred = (age_norm.item() * age_std) + age_mean
    gpred = int(gprob.argmax())
    return gpred, gprob, age_pred

# Example:
# best = ResNet18MultiTask(pretrained=False, freeze_backbone=False).to(DEVICE)
# best.load_state_dict(torch.load(resnet_ckpt, map_location=DEVICE))
# gpred, gprob, age_pred = predict_image(best, os.path.join(IMAGE_ROOT, "train/00001.jpg"))
# print(gpred, gprob, age_pred)